In [1]:
import json
import gc
import os
import pandas as pd

In [2]:
# File paths
paths = {
    'review_raw': "../data/review-California.json",
    'meta_raw': "../data/meta-California.json",
    'combined': "../data/combined_California.json",
    'cleaned_eda': "../data/cleaned_for_eda.csv",
    'cleaned_model': "../data/cleaned_for_modeling.csv"
}

# Processing chunk size
chunk_size = 1_000_000

# Process raw files pipeline

In [3]:
# Relevant columns to filter for
review_columns = ['user_id', 'time', 'rating', 'text', 'gmap_id']
meta_columns = ['gmap_id', 'name', 'category', 'avg_rating', 'num_of_reviews']

# Process reviews and extract gmap_ids
print("Processing raw review file...")
reviews_list = []
gmap_ids = set()

with open(paths['review_raw'], "r", encoding="utf-8") as fin:
    for line in fin:
        obj = json.loads(line)
        filtered_obj = {k: obj.get(k) for k in review_columns}
        reviews_list.append(filtered_obj)
        gmap_ids.add(obj["gmap_id"])

reviews = pd.DataFrame(reviews_list)
del reviews_list
gc.collect()

print(f"Reviews loaded. Unique gmap_ids: {len(gmap_ids)}")
print(f"Total reviews: {len(reviews):,}\n")

# Filter meta data
print("Processing raw meta file...")
meta_list = []

with open(paths['meta_raw'], "r", encoding="utf-8") as fin:
    for line in fin:
        obj = json.loads(line)
        if obj["gmap_id"] in gmap_ids:
            filtered_obj = {k: obj.get(k) for k in meta_columns}
            meta_list.append(filtered_obj)

meta = pd.DataFrame(meta_list)
del meta_list, gmap_ids
gc.collect()

print(f"Meta loaded. Total businesses: {len(meta):,}\n")

# Merge datasets on gmap_id
print("Merging datasets...")
df = reviews.merge(meta, on='gmap_id', how='left', suffixes=('_review', '_business'))

del reviews, meta
gc.collect()

# Rename business name column for clarity
if 'name' in df.columns:
    df = df.rename(columns={'name': 'name_business'})

print("*Combined Dataset*")
print(f"Total size: {df.shape}")
print(f"Total reviews: {len(df):,}\n")

# Function to save data in chunks
def save_in_chunks(df, output_path, chunk_size=chunk_size):
    """Save dataframe in chunks to avoid memory issues."""
    print("Saving in chunks...")
    for i in range(0, len(df), chunk_size):
        chunk = df.iloc[i:i+chunk_size]
        mode = 'w' if i == 0 else 'a'
        chunk.to_json(output_path, orient='records', lines=True, mode=mode)
        
        chunk_num = i // chunk_size + 1
        if chunk_num % 10 == 0 or i + chunk_size >= len(df):
            total_saved = min(i + chunk_size, len(df))
            print(f"Saved chunk {chunk_num}: {total_saved:,} total rows")
        
        del chunk
        gc.collect()
    print(f"Combined df saved to {output_path}")

# Save merged data
save_in_chunks(df, paths['combined'])

del df
gc.collect()
print("Processing complete.")

Processing raw review file...
Reviews loaded. Unique gmap_ids: 513134
Total reviews: 70,529,977

Processing raw meta file...
Meta loaded. Total businesses: 515,961

Merging datasets...
*Combined Dataset*
Total size: (70632313, 9)
Total reviews: 70,632,313

Saving in chunks...
Saved chunk 10: 10,000,000 total rows
Saved chunk 20: 20,000,000 total rows
Saved chunk 30: 30,000,000 total rows
Saved chunk 40: 40,000,000 total rows
Saved chunk 50: 50,000,000 total rows
Saved chunk 60: 60,000,000 total rows
Saved chunk 70: 70,000,000 total rows
Saved chunk 71: 70,632,313 total rows
Combined df saved to ../data/combined_California.json
Processing complete.


## **Column descriptions**
**Review File**  
**user_id** - ID of the reviewer  
**name** - name of the reviwer  
**time** - time of the review (unix time)  
**rating** - rating of the business  
**text** - text of the review  
**pics** - pictures of the review  
**resp** - business response to the review including unix time and text of the response  
**Meta File**  
**gmap_id** - ID of the business  
**name** - name of the business  
**address** - address of the business  
**gmap_id** - ID of the business  
**description** - description of the business  
**latitude** - latitude of the business  
**longitude** - longitude of the business  
**category** - category of the business   
**avg_rating** - average rating of the business  
**num_of_reviews** - number of reviews  
**price** - price of the business  
**hours** - open hours  
**MISC** - MISC information  
**state** - the current status of the business (e.g., permanently closed)  
**relative_results** - relative businesses recommended by Google  
**url** - URL of the business
<br>
<br>
### **Filtered these columns out**  
**'name_review'** - Reviewer name (can use id instead)  
**'pics'** - don't need  
**'resp'** - unlikely to be relevant could bring back   
**'address'** - remove for now, might extract city, state at some point  
**'description'** - mostly empty values  
**'latitude' & 'longitude'** - don't need  
**'price', 'hours', 'MISC', 'state', 'relative_results', 'url'** - not relevant  

### After merging and filtering the json was over 25GB.



# Clean data for EDA pipeline

In [4]:
# CLEANING FOR EDA CAN START HERE
# Run cells 1 & 2 first
# Then run this cell to load merged json data into df

# Load merged json data in chunks
chunks = []
print("Reading in chunks...")
for i, chunk in enumerate(pd.read_json(paths['combined'], 
                                        lines=True, 
                                        chunksize=chunk_size)):
    chunks.append(chunk)
    if (i + 1) % 10 == 0:
        total_rows = (i + 1) * chunk_size
        print(f"Loaded chunk {i+1}: {total_rows:,} total rows")

# Combine chunks to dataframe
df = pd.concat(chunks, ignore_index=True)
del chunks
gc.collect()

print(f"\nTotal merged rows: {len(df):,}")

Reading in chunks...
Loaded chunk 10: 10,000,000 total rows
Loaded chunk 20: 20,000,000 total rows
Loaded chunk 30: 30,000,000 total rows
Loaded chunk 40: 40,000,000 total rows
Loaded chunk 50: 50,000,000 total rows
Loaded chunk 60: 60,000,000 total rows
Loaded chunk 70: 70,000,000 total rows

Total merged rows: 70,632,313


In [5]:
# Basic functions to prepare data for eda
# Remove duplicates, missing ratings, missing(empty) reviews

def remove_duplicates(df):
    """Remove duplicate reviews based on user_id, gmap_id, and time."""
    print(f"Removing duplicates: {len(df):,} rows")
    duplicate_mask = df.duplicated(subset=['user_id', 'gmap_id', 'time'], keep='first')
    df_clean = df[~duplicate_mask].copy()
    print(f"  → {len(df_clean):,} rows ({duplicate_mask.sum():,} removed)\n")
    del duplicate_mask
    gc.collect()
    return df_clean

def remove_missing_ratings(df):
    """Remove rows with NaN ratings."""
    print(f"Removing missing ratings: {len(df):,} rows")
    df_clean = df.dropna(subset=['rating'])
    print(f"  → {len(df_clean):,} rows ({len(df) - len(df_clean):,} removed)\n")
    return df_clean

def remove_empty_text(df):
    """Remove reviews with NaN or empty text."""
    print(f"Removing empty text: {len(df):,} rows")
    df_clean = df[df['text'].notna() & (df['text'].str.strip() != '')]
    print(f"  → {len(df_clean):,} rows ({len(df) - len(df_clean):,} removed)\n")
    return df_clean

# Apply eda cleaning pipeline
df = (df
    .pipe(remove_duplicates)
    .pipe(remove_missing_ratings)
    .pipe(remove_empty_text)
    .reset_index(drop=True) # reset index
)

# Save eda cleaned data
df.to_csv(paths['cleaned_eda'], index=False)
print(f"Saved to {paths['cleaned_eda']}")
print()
df.info()

Removing duplicates: 70,632,313 rows
  → 69,656,135 rows (976,178 removed)

Removing missing ratings: 69,656,135 rows
  → 69,286,010 rows (370,125 removed)

Removing empty text: 69,286,010 rows
  → 38,074,228 rows (31,211,782 removed)

Saved to ../data/cleaned_for_eda.csv

<class 'pandas.DataFrame'>
RangeIndex: 38074228 entries, 0 to 38074227
Data columns (total 9 columns):
 #   Column          Dtype  
---  ------          -----  
 0   user_id         float64
 1   time            int64  
 2   rating          float64
 3   text            str    
 4   gmap_id         str    
 5   name_business   str    
 6   category        object 
 7   avg_rating      float64
 8   num_of_reviews  int64  
dtypes: float64(3), int64(2), object(1), str(3)
memory usage: 9.8+ GB


# Clean data for modeling pipeline

### Model filtering based on EDA
- Filter columns: ['user_id', 'rating', 'text', 'gmap_id', 'time']
  * rating & text are required for ABSA modeling
  * user_id & gmap_id needed for filtering, then dropped
  * time needed for temporal filtering
  * category has 86,000+ unique values, requires extensive cleaning
  * name_business, avg_rating, num_of_reviews not relevant to modeling
- Filter for users with 5-250 reviews (remove sparse and spam users)
- Filter for businesses with 10+ reviews (remove unreliable businesses)
- Filter for reviews from 2011+ (10-year recency, pre-2010 is 0.02% of data)

In [6]:
# CLEANING FOR MODEL CAN START HERE
# Run cells 1 & 2 first
# Then run this cell to cleaned for eda dataset into df

# Load cleaned dataset
print("Loading cleaned EDA dataset...")
df = pd.read_csv(paths['cleaned_eda'])
print(f"Loaded {len(df):,} rows\n")

Loading cleaned EDA dataset...
Loaded 38,074,228 rows



In [8]:
# Functions to clean data for modeling

def select_model_columns(df):
    """Select only columns needed for modeling."""
    print(f"Selecting model columns: {len(df):,} rows")
    model_cols = ['user_id', 'rating', 'text', 'gmap_id']
    df_clean = df[model_cols].copy()
    print(f"  After: {len(df_clean):,} rows (kept {len(model_cols)} columns)\n")
    return df_clean

def filter_user_review_range(df, min_reviews=5, max_reviews=250):
    """Filter for users with review count between min and max."""
    print(f"Filtering users ({min_reviews}-{max_reviews} reviews): {len(df):,} rows")
    user_counts = df['user_id'].value_counts()
    valid_users = user_counts[(user_counts >= min_reviews) & (user_counts <= max_reviews)].index
    df_clean = df[df['user_id'].isin(valid_users)].copy()
    print(f"  After: {len(df_clean):,} rows ({len(df) - len(df_clean):,} removed)\n")
    return df_clean

def filter_business_min_reviews(df, min_reviews=10):
    """Filter for businesses with minimum review count."""
    print(f"Filtering businesses ({min_reviews}+ reviews): {len(df):,} rows")
    business_counts = df['gmap_id'].value_counts()
    valid_businesses = business_counts[business_counts >= min_reviews].index
    df_clean = df[df['gmap_id'].isin(valid_businesses)].copy()
    print(f"  After: {len(df_clean):,} rows ({len(df) - len(df_clean):,} removed)\n")
    return df_clean

def filter_year_range(df, min_year=2011):
    """Filter for reviews from min_year onwards."""
    print(f"Filtering years ({min_year}+): {len(df):,} rows")
    df['year'] = pd.to_datetime(df['time'], unit='ms').dt.year
    df_clean = df[df['year'] >= min_year].copy()
    df_clean = df_clean.drop('year', axis=1)
    print(f"  After: {len(df_clean):,} rows ({len(df) - len(df_clean):,} removed)\n")
    return df_clean

# Apply model cleaning pipeline
df_model = (df
    .pipe(filter_year_range, min_year=2011)
    .pipe(select_model_columns)
    .pipe(filter_user_review_range, min_reviews=5, max_reviews=250)
    .pipe(filter_business_min_reviews, min_reviews=10)
    .reset_index(drop=True)
)

# Save model-ready data
df_model.to_csv(paths['cleaned_model'], index=False)
print(f"Saved to {paths['cleaned_model']}\n")
df_model.info()

Filtering years (2011+): 38,074,228 rows
  After: 38,053,909 rows (20,319 removed)

Selecting model columns: 38,053,909 rows
  After: 38,053,909 rows (kept 4 columns)

Filtering users (5-250 reviews): 38,053,909 rows
  After: 25,077,508 rows (12,976,401 removed)

Filtering businesses (10+ reviews): 25,077,508 rows
  After: 24,172,346 rows (905,162 removed)

Saved to ../data/cleaned_for_modeling.csv

<class 'pandas.DataFrame'>
RangeIndex: 24172346 entries, 0 to 24172345
Data columns (total 4 columns):
 #   Column   Dtype  
---  ------   -----  
 0   user_id  float64
 1   rating   float64
 2   text     str    
 3   gmap_id  str    
dtypes: float64(2), str(2)
memory usage: 4.3 GB


### Filtering Decisions for Modeling

#### 1. COLUMN SELECTION
- **Rationale:** Retained only `user_id`, `rating`, `text`, and `gmap_id` for modeling. `rating` and `text` are the core inputs and labels for ABSA. `user_id` and `gmap_id` are kept only for filtering purposes and will be dropped prior to training. `category`, `avg_rating`, `num_of_reviews`, `name_business`, and `time` were excluded as they are not direct inputs to text-based sentiment modeling.

#### 2. USER FILTERING (5-250 reviews)
- **EDA Finding:** 50% of users have only 1 review; median is 1 review per user
- **Rationale:** Single-review users likely represent one-off experiences that don't reflect consistent reviewer behavior, introducing noise into training data. A minimum of 5 reviews ensures users have enough history to reflect stable preferences. Users with 250+ reviews (0.25% of users) risk being spam accounts or automated reviewers, which could bias the model.
- **Impact:** Retains 68% of reviews (25.9M) from 14.5% of users who are consistent reviewers

#### 3. BUSINESS FILTERING (10+ reviews)
- **EDA Finding:** 34% of businesses have <10 reviews, median is 20 reviews
- **Rationale:** Low-review businesses likely lack statistical reliability for modeling. A business with 1-2 reviews doesn't provide enough signal for ABSA training.
- **Impact:** Retains 98% of reviews (37.3M), removes 34% of businesses with sparse data

#### 4. TEMPORAL FILTERING (2011+)
- **EDA Finding:** Reviews grew 5x from 2015-2016, 93% of data is 2016-2021
- **Rationale:** Google Reviews launched in 2007, added image support in 2016. Pre-2010 reviews are 0.02% of data and represent outdated platform behavior. 2011 cutoff gives us 10 years of modern review data.
- **Impact:** Removes sparse early years, focuses on consistent review platform era

#### 5. NO TEXT LENGTH FILTERING
- **EDA Finding:** Only 1.1% of reviews exceed 1000 chars, max is 13,057 chars
- **Rationale:** Longer reviews often contain more detailed sentiment and aspects. 
- **Impact:** Preserves all text data, no reviews removed by length

### Combined Impact
- **Original:** 38.1M reviews
- **After filtering:** ~24.1M reviews (63.5% retained)
- **Quality improvement:** More consistent users, reliable businesses, modern/recent data